# Libraries

In [1]:
import os

# go up one directory
os.chdir("..")

import pandas as pd 
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn_extra.cluster import KMedoids
from functions import cosmic_val
from models.denoising import *
from functions import cosmic_val
from functions import data_handling as dh
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

# set seed
# np.random.seed(15)
# torch.manual_seed(15)

# Data

In [2]:
data_path = "data/catalogues_Ovary_SBS.tsv"
cosmic_path = "data/COSMIC_v3.4_SBS_GRCh37.txt"
output_folder = "data/processed"
output_filename = "Ordered_Ovary_SBS.csv"
ordered_data_path = os.path.join(output_folder, output_filename)

In [3]:
dh.load_preprocess_data(data_path, cosmic_path, sep1 = "\t", sep2 = "\t", output_folder = output_folder, output_filename = output_filename)

Data already exists in  data/processed\Ordered_Ovary_SBS.csv


In [4]:
# load data
data = pd.read_csv(ordered_data_path, index_col = 0)
cosmic = pd.read_csv(cosmic_path, sep = "\t", index_col = 0)

# Denoising Sparse Autoencoder

In [5]:
SIGMA = 3 # standerd deviation of the noise
MU = 5 # mean of the noise
TEST_SPLIT = 0.2 # proportion of the data used for testing
BATCH_SIZE = 32 # batch size for the dataloader

In [6]:
LATENT_DIM = 4
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
CRITERION = nn.MSELoss()
EPOCHS = 5000
LEARNING_RATE = 1e-4
LAMBDA = 1e-6
CONSTRAINT = 'abs'
TOLERANCE = 1e-3

In [7]:
losses_train = []
signatures = []
iterations = 1

# transform data to tensor and add noise
data = data.T
clean_data_tensor = torch.tensor(data.iloc[:,0:].values, dtype=torch.float32)
noisy_data = add_noise(clean_data_tensor, mu = MU, sigma = SIGMA)



for i in tqdm(range(iterations)):

    # Train-test split
    num_samples = clean_data_tensor.shape[1]
    test_size = int(num_samples * TEST_SPLIT)
    train_size = num_samples - test_size

    print(num_samples)
    print(test_size)
    print(train_size)

    train_data, test_data = torch.utils.data.random_split(
        TensorDataset(noisy_data, clean_data_tensor), [train_size, test_size])

    train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True) 
    
    dsae_model = dsae(input_dim = train_size, latent_dim = LATENT_DIM, constraint=CONSTRAINT)
    # Training AENMF
    dsae_model, dsae_train_losses, dsae_val_losses, signatures_dsae, exposures_dsae = train_dsae(model = dsae_model,
                train_loader = train_loader, 
                test_loader = test_loader ,  
                criterion = CRITERION,
                optimizer = torch.optim.Adam(dsae_model.parameters(), lr=LEARNING_RATE),
                device = DEVICE, 
                epochs = EPOCHS, 
                l1_lambda = LAMBDA, 
                tol = TOLERANCE,
                relative_tol=True, 
                patience=5)    
    

    # Calculating signatures and exposures for DSAE
    diagonals_dsae = signatures_dsae.sum(axis=0)
    exposures_dsae = exposures_dsae.T @ np.diag(diagonals_dsae)
    signatures_dsae = (signatures_dsae) @ np.diag(1 / diagonals_dsae)
    
    losses_train.append(dsae_train_losses[-1])
    signatures.append(signatures_dsae)



  0%|          | 0/1 [00:00<?, ?it/s]

96
19
77


ValueError: Sum of input lengths does not equal the length of the input dataset!

In [ ]:
print("Losses train: ", np.mean(losses_train))

In [8]:
all_signatures = np.hstack(signatures)

In [9]:
pam = KMedoids(n_clusters = LATENT_DIM, metric='cosine').fit(all_signatures.T)
consensus_signatures = all_signatures[:, pam.medoid_indices_]

In [10]:
matched_signatures, mean_similarity = cosmic_val.compute_match(consensus_signatures, cosmic)

In [ ]:
print(matched_signatures.head())
print("\nMean similarity of the matched signatures: ", mean_similarity)